# Projeto de Análise de Erro

Esse projeto busca abordar erros associados a representação computacional de ponto flutuante, na execução de operações de produto interno de vetores N-dimensionais. Para fins de análise, utilizaremos o protudo interno usual definido abaixo:

\begin{equation}
<X, Y> = \sum_{i=1}^{n} x_i y_i = x_1 y_1 + x_2 y_2 + ... + x_n y_n
\end{equation}

Para realizarmos o estudo do impacto do erro de máquina sobre esta operação, desvemos possuir um valor alvo esperado, esse valor ideal representa o resultado desejado ao final da operação, onde a diferença entre o valor real obtido e o valor ideal é o erro gerado durante sua execução. Para obter esse valor ideal, iremos considerar que os vetores nessa análise estão normalizados, isso é, para todo vetor $X$, sua norma é iqual a $1$, logo:

\begin{equation}
||X|| = \sqrt{<X, X>} = <X, X> = 1
\end{equation}

Com o objetivo de definir um escopo mais restrito ao projeto, também iremos considerar que todos os vetores são binários, isso é, para todo vetor $X$, $x_i = 0$ ou $x_i = k$, onde $0 <= i < N$ e $k \in R$. Vale resaltar que este tipo de configuração de vetores é bem comum para representação de dados, como palavras, textos, documentos, taxonomias, classificação, etc.

Este projeto será execultado na linguagem C/C++, perminitindo interpretar mais facilmente os problemas associados aos bits, além disso será ultilizado a representação de ponto flutuante com menor precisão em C/C++, neste caso o "_float_", trata-se de uma representação padronizada pela _IEEE_ com 32 bits nomeada de _single_ (a representação de 16 bits (_half_) não é nativa em C/C++). Por padrão, C/C++ utiliza truncamento na adminstração de dados.

Resumindo, teremos:

> - Operação de produto interno como alvo da ánalise de erro.
> - Todos os vetores que envolvem o problema possuem norma 1, a norma será o valor esperado do produto interno.
> - Todos os vetores são binários, possuindo apenas dois valores possíveis em cada dimensão, reduzindo o escopo do problema.
> - Representação _float_ em C/C++ com 32 bits de precisão com trucamento.

## Procedimentos Iniciais

Para analizar os dados visualmente, iremos importar as bibliotecas gráficas utilizadas no jupyter e no interpretador de C/C++, neste caso, o Xplot da Xeus, que usa como base o bgplot em Python. Também seram iniciados as estruturas de dados padrão que seram utilizadas durante toda a análise.

In [1]:
#include "xplot/xfigure.hpp"
#include "xplot/xmarks.hpp"
#include "xplot/xaxes.hpp"

using namespace std;

vector<double> x;
vector<double> y;

## Implementação Singela de Produto Interno

Como desejamos simular o produto interno para uma situação muito especifica, isso é, para vetores binários normalizados aplicados a relação $<X, X> = 1$, podemos partir do principio de algumas otimizações que não impactam na implementação singela de produto interno, tais como:

> - Produto interno é indiferente quanto a dimensões nulas, podendo ser retiradas da operação.
> - A dimensão do vetor é irrelevante, desde que o número de dimensões não-nulas seja informado.
> - não é necessario a costrução de uma estrutura de array para representação do vetor, por ele ser binário.

In [2]:
void dotNaive(int size, int shift=0){
    x = vector<double>(size);
    y = vector<double>(size);
    x[0] = 0; y[0] = 1.0;
    for(int i=1 ; i<size ; i++){
        int aux = i << shift;
        float d = 1.0 / sqrt(aux);
        float sum = 0.0f;
        for(int j=0 ; j<aux ; j++){
            sum += d*d;
        }
        x[i] = aux;
        y[i] = sum;
    }
}

## Primeiro teste de execução

Iremos executar o código para gerar o resultado dos 10000 primeiros vetores com dimensão não-nula até 10000, o resultado segue no gráfico abaixo:

In [3]:
dotNaive(10000);
xpl::linear_scale sx1, sy1;
auto fig1 = xpl::figure_generator().padding_x(0.025).padding_y(0.025).finalize();
fig1.add_mark(xpl::scatter_generator(sx1, sy1).x(x).y(y).default_size(1).finalize());
fig1.add_axis(xpl::axis_generator(sy1).label("Valor Obtido").orientation("vertical").side("left").finalize());
fig1.add_axis(xpl::axis_generator(sx1).label("Número de Dimensões Não-Nulas").finalize());
fig1

A Jupyter widget

## Análise de resultados 

Do gráfico acima podemos observar que ocorrereu grande dispersão do valor com relação ao valor esperado, observe também que o erro associado cresce significativamente conforme o número de dimensões não-nulas crescem, demostrando uma forte correlação desta caracteristica com o erro. Observe tambem que o erro parece ser mais intenso em duas regioes especificas demostrando uma configuração "bi-gaussiana", sendo menos intenso com valores proximos ao ideial, e menos recorrente nos erros extremos, contudo elevado nos valores de erro medio absoluto. Observe também que o gráfico aparenta demostrar determinados padrões de frequência, como pequenas senoídes, irremos ataca esse padrão de forma a evidencia-lo e demostrar suas características, para isso iremos aplica o produto vetorial em vetores com potencias de 2 elementos não-nulos afim de reforçar a ocorrência do padrão. 

In [4]:
dotNaive(10000, 8);
xpl::linear_scale sx2, sy2;
auto fig2 = xpl::figure_generator().padding_x(0.025).padding_y(0.025).finalize();
fig2.add_mark(xpl::scatter_generator(sx2, sy2).x(x).y(y).default_size(1).finalize());
fig2.add_axis(xpl::axis_generator(sy2).label("Valor Obtido").orientation("vertical").side("left").finalize());
fig2.add_axis(xpl::axis_generator(sx2).label("Número de Dimensões Não-Nulas").finalize());
fig2

A Jupyter widget

In [6]:
dotNaive(200, 23);
xpl::linear_scale sx3, sy3;
auto fig3 = xpl::figure_generator().padding_x(0.025).padding_y(0.025).finalize();
fig3.add_mark(xpl::scatter_generator(sx3, sy3).x(x).y(y).default_size(1).finalize());
fig3.add_axis(xpl::axis_generator(sy3).label("Valor Obtido").orientation("vertical").side("left").finalize());
fig3.add_axis(xpl::axis_generator(sx3).label("Número de Dimensões Não-Nulas").finalize());
fig3

A Jupyter widget

In [4]:
xpl::linear_scale xs, ys;
xpl::hist hist(xs, ys);
for(int i=0 ; i<size ; i++){
    y_data[i] = abs(1 - y_data[i]);
}
hist.sample = y_data;

In [5]:
xpl::figure fig2;
fig2.add_mark(hist);
fig2

A Jupyter widget

In [6]:
xpl::axis hx(xs), hy(ys);
hy.orientation = "vertical";
fig2.add_axis(hx);
fig2.add_axis(hy);
hist.colors = std::vector<std::string>{"21c0fc"};